## Resource-Constrained Project Scheduling Problem with Alternative Subgraphs
### Simplified Formulation

This notebook demonstrates a simplified formulation of the Resource-Constrained Project Scheduling Problem with Alternative Subgraphs (RCPSP-AS). The formulation is based on the work by Servranckx & Vanhoucke (2019) but uses a streamlined constraint set that eliminates auxiliary structures.

### Problem Definition

The RCPSP-AS models a project as a directed acyclic graph $G=(N,A)$, where:

- $N = \{1, \dots, n\}$ — topologically ordered activities
- $A \subset N \times N$ — precedence arcs
- $d_i$ — duration of activity $i$
- $r_{i,v}$ — demand of activity $i$ for resource $v \in R$
- $a_v$ — capacity of renewable resource $v$
- $L$ — set of alternative subgraphs
- $p_l$ — principal (branching) activity of subgraph $l \in L$

The immediate successors of each principal activity $p_l$ represent mutually exclusive branches. The key derived set is:

$$A_{\text{prop}} = A \setminus \{(p_l, j) \in A : l \in L\}$$

This represents all arcs **except** the branching arcs from principal activities.

### Simplified CP Formulation

$$
\begin{aligned}
\min \quad
& \mathrm{endOf}(\mathrm{x}_{n-1})
\qquad &\qquad & \text{(1)} \\[2mm]
\text{s.t.} \quad
& \mathrm{presenceOf}(\mathrm{x}_{0}) = 1,
\qquad &\qquad & \text{(2)} \\[1mm]
& \mathrm{presenceOf}(\mathrm{x}_i) \Rightarrow \mathrm{presenceOf}(\mathrm{x}_j),
\qquad & \forall (i,j)\in A_{\text{prop}}
\quad & \text{(3)} \\[1mm]
& \sum_{j:\, (p_l, j) \in A} \mathrm{presenceOf}(\mathrm{x}_j) = \mathrm{presenceOf}(\mathrm{x}_{p_l}),
\qquad & \forall l \in L
\quad & \text{(4)} \\[1mm]
& \mathrm{ifThen}\!\left(\mathrm{presenceOf}(\mathrm{x}_i) \wedge \mathrm{presenceOf}(\mathrm{x}_j),\; \mathrm{endOf}(\mathrm{x}_i) \le \mathrm{startOf}(\mathrm{x}_j)\right),
\qquad & \forall (i,j)\in A
\quad & \text{(5)} \\[1mm]
& \sum_{i \in N} \mathrm{pulse}(\mathrm{x}_i, r_{i,v}) \le a_v,
\qquad & \forall v \in R
\quad & \text{(6)} \\[1mm]
& \text{interval } \mathrm{x}_i\ \text{optional, size}=d_i,
\qquad & \forall i \in N
\quad & \text{(7)}
\end{aligned}
$$

**Objective:**

* **(1)** Minimize the makespan $C_{\max}$, defined as the end time of the sink activity $n-1$.

**Modeling Constraints:**

* **(2)** Source activity is always selected (project begins)
* **(3)** **Selection propagation**: If activity $i$ is selected and $(i,j)$ is a non-branching arc, then $j$ must be selected. This automatically handles fixed activities without explicit enumeration.
* **(4)** **Branch selection**: For each alternative subgraph $l$, exactly one successor of the principal activity $p_l$ is selected if and only if $p_l$ is selected.
* **(5)** **Precedence timing**: If both activities $i$ and $j$ are present, then $i$ must finish before $j$ starts.
* **(6)** **Resource limits**: The sum of resource demands from all active intervals must not exceed capacity.

**Variables:**

* **(7)** $\mathrm{x}_i$: An optional interval variable for each activity $i \in N$ with a fixed duration $d_i$.

#### Symbols and Notation

| Symbol / Function | Meaning | `docplex.cp` Reference | OptalCP Reference |
| :--- | :--- | :--- | :--- |
| $N$ | Set of activities, indexed $0..n-1$ | — | — |
| $A$ | Set of precedence constraints $(i,j)$ | — | — |
| $A_{\text{prop}}$ | Non-branching arcs | — | — |
| $L$ | Set of alternative subgraphs | — | — |
| $R$ | Set of renewable resources | — | — |
| $d_i$ | Duration of activity $i$ | — | — |
| $r_{i,v}$ | Demand of activity $i$ for resource $v$ | — | — |
| $a_v$ | Capacity of renewable resource $v$ | — | — |
| $p_l$ | Principal activity of subgraph $l$ | — | — |
| $\mathrm{x}_i$ | Optional interval variable for activity $i$ | [interval_var](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.interval_var) | [interval_var](http://dev.vilim.eu/python-api/api.html#optalcp.Model.interval_var) |
| $\mathrm{presenceOf}(\mathrm{x}_i)$ | $1$ if interval $\text{x}_{i}$ is present, else $0$ | [presence_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.presence_of) | [interval_var.presence](http://dev.vilim.eu/python-api/api.html#optalcp.IntervalVar.presence) |
| $\mathrm{endOf}(\dots)$ | End time of an interval | [end_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_of) | [interval_var.end](http://dev.vilim.eu/python-api/api.html#optalcp.IntervalVar.end) |
| $\mathrm{startOf}(\dots)$ | Start time of an interval | [start_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.start_of) | [interval_var.start](http://dev.vilim.eu/python-api/api.html#optalcp.IntervalVar.start) |
| $\mathrm{ifThen}(\dots)$ | Logical implication constraint | [if_then](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.if_then) | [bool_expr.implies](http://dev.vilim.eu/python-api/api.html#optalcp.BoolExpr.implies) |
| $\mathrm{pulse}(\dots)$ | Time-varying resource usage | [pulse](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.pulse) | [interval_var.pulse](http://dev.vilim.eu/python-api/api.html#optalcp.IntervalVar.pulse) |

### Data Import & Parsing

In [47]:
%pip install -q docplex graphviz ortools
from io import StringIO
import graphviz as gv

# Using ascp package from Charvat's bachelor thesis
from ascp.load_instance import load_instance
from ascp.instance import AslibInstance, Subgraph
from ascp.graphviz import show_instance
from ascp.solver import Solution, SolvedActivity

Note: you may need to restart the kernel to use updated packages.


#### Reading the data file

In [48]:
instance = load_instance('../../data/rcpspas/ASLIB/ASLIB0/aslib0_25678a.RCP')

# Create a dictionary to map activity IDs (0-based) to their objects
act_dict = {act.id: act for act in instance.activities}

# Build set of branching arcs (from principal activities to their successors)
# These arcs are NOT in A_prop
branching_arcs = {
    (sub.principal_activity, s)
    for sub in instance.subgraphs if sub.principal_activity in act_dict
    for s in act_dict[sub.principal_activity].successors if s in act_dict
}

#### Visualize the instance structure

In [49]:
dot_output = StringIO()
show_instance(instance, file=dot_output)
dot_string = dot_output.getvalue()

graph = gv.Source(dot_string)
# display(graph)

### IBM CPO DOcplex Implementation

#### Imports

In [50]:
from docplex.cp.model import CpoModel

#### Create model and variables

In [51]:
# Create a CP Optimizer model
mdl = CpoModel(name=instance.name)

# (7) Create optional interval variable for each activity i with a fixed duration d_i
x = {i: mdl.interval_var(
    name=f"T_{i}", optional=True, size=act.duration) for i, act in act_dict.items()}

#### Add constraints and define objective

In [52]:
# (1) Minimize the makespan (end time of the single sink activity)
mdl.add(mdl.minimize(mdl.end_of(x[len(instance.activities) - 1])))

# (2) Source activity is present
mdl.add(mdl.presence_of(x[0]) == 1)

# (3) Selection propagation along A_prop (non-branching arcs)
for act in instance.activities:
    for j in act.successors:
        if j in x and (act.id, j) not in branching_arcs:
            # If activity i is selected, then j must be selected
            mdl.add(mdl.presence_of(x[act.id]) <= mdl.presence_of(x[j]))

# (4) Branch selection for each subgraph
for sub in instance.subgraphs:
    if sub.principal_activity in act_dict:
        successors = [s for s in act_dict[sub.principal_activity].successors if s in x]
        if successors:
            mdl.add(mdl.sum(mdl.presence_of(x[s]) for s in successors) ==
                   mdl.presence_of(x[sub.principal_activity]))

# (5) Precedence timing for all arcs
mdl.add(mdl.if_then(mdl.presence_of(x[act.id]) & mdl.presence_of(x[j]),
                    mdl.end_of(x[act.id]) <= mdl.start_of(x[j]))
        for act in instance.activities for j in act.successors if j in x)

# (6) Resource capacity constraints
mdl.add(mdl.sum(mdl.pulse(x[act.id], act.requirements[v])
                for act in instance.activities if act.requirements[v] > 0) <= capacity
        for v, capacity in enumerate(instance.resources) if capacity > 0)

#### Solve the model

In [ ]:
from docplex.cp.model import CpoModel
from docplex.cp.parameters import CpoParameters

# =============================================================================
# IBM CP Optimizer Parameters for RCPSP-AS (Proving Optimality)
# =============================================================================

params = CpoParameters()

# --- Basic Settings ---
params.Workers = 8
params.TimeLimit = 100
params.LogVerbosity = 'Normal'
params.LogPeriod = 5000

# --- Search Strategy ---
params.SearchType = 'Restart'             # Works well with FDS

# --- Failure-Directed Search (Key for proving bounds!) ---
params.FailureDirectedSearch = 'On'
params.FailureDirectedSearchEmphasis = 5  # 5+ workers dedicated to FDS
params.FailureDirectedSearchMaxMemory = 500000000  # 500MB (increase from 100MB default)

# --- Inference Levels (Propagation Strength) ---
# Extended = maximum propagation, critical for proving optimality

# General default
params.DefaultInferenceLevel = 'Extended'

# Cumulative functions (pulse constraints for resources)
params.CumulFunctionInferenceLevel = 'Extended'

# Precedence constraints (critical for scheduling)
params.PrecedenceInferenceLevel = 'Extended'

# Interval sequences (if you have sequence variables)
params.IntervalSequenceInferenceLevel = 'Extended'

# --- Solve ---
print('Solving IBM CP Optimizer model...')
solution = mdl.solve(params=params)
print('Solution:')
solution.print_solution()

Solving IBM CP Optimizer model...
Solution:


### OptalCP Implementation

#### Imports

In [55]:
import optalcp as cp

#### Create model and variables

In [56]:
# Create an OptalCP model
mdl_ocp = cp.Model()

# (7) Create optional interval variable for each activity i with a fixed duration d_i
x_ocp = {i: mdl_ocp.interval_var(
    name=f"T_{i}", optional=True, length=act.duration) for i, act in act_dict.items()}

#### Add constraints and define objective

In [ ]:
# (1) Minimize makespan
mdl_ocp.minimize(x_ocp[len(instance.activities) - 1].end())

# (2) Source activity is present
mdl_ocp.enforce(x_ocp[0].presence() == 1)

# (3) Selection propagation along A_prop (non-branching arcs)
for act in instance.activities:
    for j in act.successors:
        if j in x_ocp and (act.id, j) not in branching_arcs:
            # If activity i is selected, then j must be selected
            mdl_ocp.enforce(x_ocp[act.id].presence().implies(x_ocp[j].presence()))

# (4) Branch selection for each subgraph
for sub in instance.subgraphs:
    if sub.principal_activity in act_dict:
        successors = [s for s in act_dict[sub.principal_activity].successors if s in x_ocp]
        if successors:
            mdl_ocp.enforce(sum(x_ocp[s].presence() for s in successors) == 
                       x_ocp[sub.principal_activity].presence())

# (5) Precedence timing for all arcs
for act in instance.activities:
    for j in act.successors:
        if j in x_ocp:
            mdl_ocp.enforce((x_ocp[act.id].presence() & x_ocp[j].presence()).implies(
                x_ocp[act.id].end() <= x_ocp[j].start()))

# (6) Resource capacity constraints
for v, capacity in enumerate(instance.resources):
    if capacity > 0:
        pulses = [x_ocp[act.id].pulse(height=act.requirements[v])
                 for act in instance.activities if act.requirements[v] > 0]
        if pulses:
            mdl_ocp.enforce(mdl_ocp.sum(pulses) <= capacity)

#### Solve the model

In [ ]:
import optalcp as cp

# =============================================================================
# OptalCP 2026.1.0 Parameters for RCPSP-AS (Proving Optimality)
# =============================================================================

params = cp.Parameters()

# --- Basic Settings ---
params["nbWorkers"] = 8
params["timeLimit"] = 100
params["logLevel"] = 2
params["logPeriod"] = 5

# --- Propagation Levels (Key for this problem!) ---
# Cumulative resource constraints (pulse functions)
params["cumulPropagationLevel"] = 3          # Max level for cumul constraints

# Precedence propagation (critical for RCPSP)
params["usePrecedenceEnergy"] = 1            # Enable precedence energy algorithm

# Position propagation (helps with optional intervals)
params["positionPropagationLevel"] = 3       # Higher propagation for positions

# Reservoir (for step functions if used)
params["reservoirPropagationLevel"] = 2

# --- Lower Bound Computation ---
params["simpleLBShavingRounds"] = 5          # Aggressive domain shaving
params["simpleLBMaxIterations"] = 2147483647 # No limit on LB iterations

# --- Worker Configuration (Hybrid Search) ---
# Mix of LNS (solution finding) and FDS/FDSDual (bound proving)

lns_worker = {"searchType": "LNS"}

fds_worker = {
    "searchType": "FDS",
    "cumulPropagationLevel": 3,
    "positionPropagationLevel": 3
}

fdsdual_worker = {
    "searchType": "FDSDual",  # Renamed from FDSLB in 2026.1.0
    "cumulPropagationLevel": 3,
    "positionPropagationLevel": 3
}

# Distribution: 2 LNS, 2 FDS, 4 FDSDual (emphasize bound proving)
params["workers"] = [
    lns_worker,
    lns_worker,
    fds_worker,
    fds_worker,
    fdsdual_worker,
    fdsdual_worker,
    fdsdual_worker,
    fdsdual_worker,
]

# --- Solve ---
print('Solving OptalCP model...')
result_ocp = mdl_ocp.solve(parameters=params)
print('OptalCP Solution:')
print(result_ocp)
print(f'Optimal: {result_ocp.proof}')

## Additional Resources

- [ASLIB dataset](https://www.projectmanagement.ugent.be/research/data)
- [Jakub Charvat bachelor thesis](https://dspace.cvut.cz/handle/10467/123532)
- [Servranckx & Vanhoucke (2019) - Original RCPSP-AS formulation](https://doi.org/10.1016/j.ejor.2018.09.005)